In [27]:
import openpyxl
import math as ma
p=[i for i in range(1,38)]
n=[i for i in range(1,38)]
P=[i for i in range(37)]
N=[i for i in range(37)]

book1 = openpyxl.load_workbook("Block location (5).xlsx")
sh=book1["NC2NC"]

Potential_national_facility=[]
for i in p:
    Potential_national_facility.append(sh.cell(0+1,i+1).value)

dn2=[]
dn2n=[]
for i in p:
    for j in n:
        dn2.append(int(sh.cell(i+1,j+1).value))
    dn2n.append(dn2)
    dn2=[]
  
Dn2n=[]
ND=book1["NC Population"] #NC Demand
for i in Potential_national_facility:
    for j in range(1,38):
        if i==str(ND.cell(j+1,0+1).value):
            Dn2n.append(int(ND.cell(j+1,2+1).value))

In [28]:
from gurobipy import Model, GRB, quicksum
import gurobipy as gp
from itertools import product

m=gp.Model('Facility_Centre')
y=m.addVars(P,N,vtype=GRB.BINARY,name='Y')
x=m.addVars(P,vtype=GRB.BINARY,name='X')

m.addConstrs((gp.quicksum(y[i,j] for i in P)==1 for j in N));
m.addConstr((gp.quicksum(x[i] for i in P)==5));
m.addConstrs((y[i,j] <= x[i] for j in N for i in P))


vj=[45104, 11275, 1556487, 197530, 46000, 1556487, 13088, 13088, 13088, 768118, 27813, 18542, 22662, 33993, 235042, 737107, 55625, 384842, 28051, 17446, 425766, 5563, 9845, 157612, 9271, 1056513, 1070383, 6400, 48816, 17446, 34766, 327676, 60500, 14278, 695592, 25495, 211011]

m.setObjective(gp.quicksum(dn2n[i][j] * Dn2n[j]/vj[j] * y[i,j] for j in N for i in P ),GRB.MINIMIZE)

m.Params.Cuts=0
m.Params.Heuristics=0
m.Params.Presolve=0
m.Params.Symmetry=0
m.Params.RINS=0
m.Params.Aggregate=0
m.Params.Disconnected=0
m.optimize()

TNC=[]
for i in P:
    if x[i].X==1:
        TNC.append(Potential_national_facility[i])
    else:
        continue
Arcs=[]
for i in P:
    for j in N:
        if y[i,j].X == 1:
            Arcs.append(Potential_national_facility[i]+" : "+Potential_national_facility[j])

Arcs1={}
list5=[]
for i in P:
    for j in N:
        if y[i,j].X == 1:
            list5.append(Potential_national_facility[j])
    if len(list5)!=0:
        Arcs1[str(Potential_national_facility[i])]=list5
        list5=[]
    else:
        continue
#Distance from transshipment National Centres to State Capitals
dtn2n=[]
for i in TNC:
    if i in Potential_national_facility:
        dtn2n.append(dn2n[Potential_national_facility.index(i)])

Set parameter Cuts to value 0
Set parameter Heuristics to value 0
Set parameter Presolve to value 0
Set parameter Symmetry to value 0
Set parameter RINS to value 0
Set parameter Aggregate to value 0
Set parameter Disconnected to value 0
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 5800HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 1407 rows, 1406 columns and 4144 nonzeros
Model fingerprint: 0x966e2598
Variable types: 0 continuous, 1406 integer (1406 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+02, 7e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Variable types: 0 continuous, 1406 integer (1406 binary)

Root relaxation: objective 2.054159e+06, 250 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds   

In [29]:
Manufacturers=['Serum Institute of India, Pune','Bharat Biotech International Ltd, Hyderabad']#,'Cadila Healthcare Ltd, Ahmedabad','Trialed and manufactured in India by Dr. Reddy Lab.','Serum Institute of India, Pune']#,'Biological E Ltd, Hyderabad','Genova, Pune','Bharat Biotech International Ltd, Hyderabad','Aurobindo Pharma Ltd, Hyderabad']
Suppliers=['A','B']#,'C','D']
V=[300, 25, 50]
PCapr=[475000,15000,5]
Wijrm2n=[]
Wij=[]
wi=[]
NDA=book1["M2NAir"] #Accessing the data of manufacturers to Distance from excel sheet 
NDR=book1["M2NRoad"]
NDM=book1["M2NMotor"]
for i in range(1,len(Potential_national_facility)+1):
    for j in range(1,len(Potential_national_facility)+1):
        if (NDA.cell(i+1,0+1).value in Manufacturers or NDA.cell(i+1,0+1).value in Suppliers) and NDA.cell(0+1,j+1).value in TNC:
            wi.append(int(NDA.cell(i+1,j+1).value))
            wi.append(int(NDR.cell(i+1,j+1).value))
            wi.append(int(NDM.cell(i+1,j+1).value))
            if len(wi)!=0:
                Wij.append(wi)
                wi=[]
            else:
                print(wi)
    if len(Wij)!=0:        
        Wijrm2n.append(Wij)
        Wij=[]
    else:
        continue

dM2=[]
dM2T=[]
dM2TN=[]
M2NA=book1["M2TNCA"] #Accessing the data of manufacturers to Distance from excel sheet 
M2N=book1["M2NC"]
for i in range(50):
    for j in range(len(Potential_national_facility)+1):
        if (M2NA.cell(i+1,0+1).value in Manufacturers or M2NA.cell(i+1,0+1).value in Suppliers) and M2NA.cell(0+1,j+1).value in TNC:
            dM2.append(int(M2NA.cell(i+1,j+1).value))
            dM2.append(int(M2N.cell(i+1,j+1).value))
            dM2.append(int(M2N.cell(i+1,j+1).value))
            if len(dM2)!=0:
                dM2T.append(dM2)
                dM2=[]
    if len(dM2T)!=0:        
        dM2TN.append(dM2T)
        dM2T=[]
    else:
        continue
Dm2={}
list2=[]
for i in P:
    for j in N:
        if y[i,j].X == 1:
            list2.append(Dn2n[j])
    if len(list2)!=0:
        Dm2[str(Potential_national_facility[i])]=list2
        list2=[]
    else:
        continue

Dm2n={}
for i in Dm2:
    Dm2n[i]=sum(Dm2[i])

DM2N=[]
for i in Dm2n:
    DM2N.append(Dm2n[i])


In [30]:
DM2NP=[]
DM2NP.append(DM2N)
VR=0.3
M=[i for i in range(len(Manufacturers))]
P=[i for i in range(5)]
print(P)
for p in P:
    if p>(0):
        DM2NP.append([int(i*(1-1/(p+1))) for i in DM2N])

[0, 1, 2, 3, 4]


In [31]:
# Define ranges and sets
M = [i for i in range(len(Manufacturers))]
S = [i for i in range(len(M), len(Manufacturers) + len(Suppliers))]
N = [i for i in range(len(TNC))]
T = M + S
TD = 0
R = [0, 1, 2]
F = [0, 1, 2] 

# capacities and costs
CapR = [400000000, 200000000, 10000000]
CF = [[158191, 15000, 5000] for j in N]

# Initialize matrices to zero and assign values conditionally
MCap = [[[0 for t in T] for i in M + S] for p in P]
Mexp = [[[0 for t in T] for i in M + S] for p in P]
SCap = [[[0 for t in T] for i in M + S] for p in P]

for p in P:
    for i in M + S:
        for t in T:
            if i == t:
                if i < len(M):  # For manufacturers
                    MCap[p][i][t] = int(120000000)
                    Mexp[p][i][t] = int(2000000)
                else:  # For suppliers
                    SCap[p][i][t] = int(1000000)

# Check results for MCap of the first period
MCap[0]


[[120000000, 0, 0, 0], [0, 120000000, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]

In [32]:
Cost=[5,0.54,0.23]
ex=3
rv=2
pd = 0.8
rpd=0.7

In [33]:
f1=0.3
f2=1-f1
DM2NPPP=[]
DM2NPPP.append(DM2N)
PctP=[0 for i in P]
PctP[0]=0.99999*((sum(sum(MCap[0][i][t] for t in T) for i in M)-sum(sum(Mexp[0][i][t] for t in T) for i in M)+sum(sum(SCap[0][i][t] for t in T) for i in S))/(sum(DM2NPPP[0])))
for p in range(1,len(P)):
    a=[]
    for j in N:
        a.append(int(DM2NPPP[p-1][j]*(1-PctP[p-1])))
    DM2NPPP.append(a)
    if p<rv:
        PctP[p]=0.99999*(((sum(sum(MCap[p][i][t]for t in T) for i in M)-sum(sum(Mexp[p][i][t]for t in T) for i in M)+sum(sum(SCap[p][i][t]for t in T) for i in S))/(sum(DM2NPPP[p]))))
    else:
        PctP[p]=0.99999*((f1*(sum(sum(MCap[p][i][t]for t in T) for i in M)-sum(sum(Mexp[p][i][t]for t in T) for i in M)+sum(sum(SCap[p][i][t]for t in T) for i in S))/(sum(DM2NPPP[p]))))
PctP

[0.19659970422864745,
 0.24470952508963476,
 0.09719817760906015,
 0.10766280665156337,
 0.12065260531544256]

# model for regular vaccination

In [34]:
m1=gp.Model('IEV_M1')

import math as ma

XMijtp=m1.addVars(M,N,T,P, vtype=GRB.CONTINUOUS, name='XMijtp')
XIijtp=m1.addVars(S,N,T,P, vtype=GRB.CONTINUOUS, name='XIijtp')

TRijp=m1.addVars(M+S,N,P, vtype=GRB.INTEGER, name='TRijp')
Yijrp=m1.addVars(M+S,N,R,P, vtype=GRB.BINARY, name='Yijrp')
neta=m1.addVars(N,F,vtype=GRB.BINARY, name='neta')


m1.addConstrs(gp.quicksum(XMijtp[i,j,t,p] for j in N) <=(MCap[p][i][t])-(Mexp[p][i][t]) for p in range(0,len(P)) for i in M for t in T)

m1.addConstrs(gp.quicksum(XIijtp[i,j,t,p] for j in N)<=SCap[p][i][t] for p in range(0,len(P)) for i in S for t in T)

m1.addConstrs(gp.quicksum(gp.quicksum(XMijtp[i,j,t,p] for i in M for j in N)+gp.quicksum(XIijtp[i,j,t,p] for i in S for j in N)for t in T)
              <=f1*(sum(sum(MCap[p][i][t]for t in T) for i in M)-
                    sum(sum(Mexp[p][i][t]for t in T) for i in M)+sum(sum(SCap[p][i][t]for t in T)for i in S))for p in range (rv, len(P)))

Vac_avai_jtp=[[[0 for p in P] for t in T] for j in N]

m1.addConstrs(Wijrm2n[i][j][r]-Yijrp[i,j,r,p]>=0 for i in M+S for j in N for r in R for p in range(0,len(P)))

m1.addConstrs(Wijrm2n[i][j][r]-Yijrp[i,j,r,p]>=0 for i in M+S for j in N for r in R for p in range(0,len(P)))

for j in N:
    for t in T:
        for p in P:
            if p<rv:
                Vac_avai_jtp[j][t][p]=gp.quicksum(((1-pd)**(p-k))*(gp.quicksum(XMijtp[i,j,t,k]for i in M)+gp.quicksum(XIijtp[i,j,t,k] for i in S)) for k in range(max(0,p-ex), p+1))
            if p>=rv:
                Vac_avai_jtp[j][t][p]=gp.quicksum(((1-pd)**(p-k))*(gp.quicksum(XMijtp[i,j,t,k] for i in M)+gp.quicksum(XIijtp[i,j,t,k] for i in S)) for k in range(max(0,p-ex), p+1))

m1.addConstrs(gp.quicksum(Vac_avai_jtp[j][t][p] for t in T) <= gp.quicksum(CapR[f]*neta[j,f] for f in F) for j in N for p in P)

m1.addConstrs(gp.quicksum(XMijtp[i,j,t,p] for i in M for t in T) + gp.quicksum(XIijtp[i,j,t,p] for i in S for t in T)>=ma.ceil(PctP[p]*(DM2NPPP[p][j])) for p in range(0,len(P)) for j in N)

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,


In [35]:
wijrp= m1.addVars(M+S, N, R, P, vtype=GRB.CONTINUOUS, name="wijrp")

m1.addConstrs(gp.quicksum(PCapr[r]*wijrp[i,j,r,p] for r in R)-gp.quicksum((XMijtp[i,j,t,p]) for t in T)*0.005 >=0 for j in N for i in M for p in range(0,len(P)))
m1.addConstrs(gp.quicksum(PCapr[r]*wijrp[i,j,r,p] for r in R)-gp.quicksum((XIijtp[i,j,t,p]) for t in T )*0.005 >=0 for j in N for i in S for p in range(0,len(P)))

m1.addConstrs(wijrp[i,j,r,p]>=TRijp[i,j,p]-200*(1-Yijrp[i,j,r,p]) for r in R for j in N for i in M+S for p in range(0,len(P)))
m1.addConstrs(gp.quicksum(wijrp[i,j,r,p] for r in R)<=TRijp[i,j,p] for p in P for i in M+S for j in N)

m1.addConstrs(wijrp[i,j,r,p]<=200* Yijrp[i,j,r,p] for p in range(0,len(P)) for i in M+S for j in N for r in R)

{(0, 0, 0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 4, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 4, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 0, 2): <gurobi.Constr *Awaiting Model Up

In [36]:
TC=gp.quicksum(2*dM2TN[i][j][r]*Cost[r]*TRijp[i,j,p]for p in range(0,len(P)) for r in R for j in N for i in M+S)
MC=gp.quicksum(neta[j,f]*CF[j][f] for j in N for f in F)
HC=gp.quicksum(Vac_avai_jtp[j][t][p]-
               (gp.quicksum(XMijtp[i,j,t,p] for i in M)+
                gp.quicksum(XIijtp[i,j,t,p] for i in S)
               )for j in N for t in T for p in P)
CH=gp.quicksum(Cost[r]*Yijrp[i,j,r,p] for p in P for i in M+S for j in N for r in R)
m1.setObjective(TC+MC+HC+CH, GRB.MINIMIZE)

In [37]:
m1.Params.Heuristics=0
m1.Params.Presolve=0
m1.setParam("IntegralityFocus",1)
m1.setParam("IntFeasTol",1e-09)
m1.setParam("MIPGap", 0.0005)
m1.optimize()

Set parameter Heuristics to value 0
Set parameter Presolve to value 0
Set parameter IntegralityFocus to value 1
Set parameter IntFeasTol to value 1e-09
Set parameter MIPGap to value 0.0005
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 5800HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 1533 rows, 1115 columns and 5435 nonzeros
Model fingerprint: 0xc4122a33
Variable types: 700 continuous, 415 integer (315 binary)
Coefficient statistics:
  Matrix range     [5e-03, 4e+08]
  Objective range  [2e-01, 2e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+08]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Variable types: 700 continuous, 415 integer (315 binary)

Root relaxation: objective 1.495410e+08, 646 iterations, 0.01 seconds (0.00 work units)


In [38]:
# for p in range(0,len(P)):
#     for t in T:
#         for j in N:
#             for i in M:
#                 if XMijtp[i,j,t,p].X!=int(XMijtp[i,j,t,p].X):
#                     print('XM',i,j,t,p) 
#                     print(XMijtp[i,j,t,p].X-int(XMijtp[i,j,t,p].X))
#                 print(XMijtp[i,j,t,p].X==int(XMijtp[i,j,t,p].X))

                
# for p in range(0,len(P)):
#     for t in T:
#         for j in N:
#             for i in S:
#                 if XIijtp[i,j,t,p].X!=int(XIijtp[i,j,t,p].X):
#                     print('XI', i,j,t,p) 
#                     print(XIijtp[i,j,t,p].X-int(XIijtp[i,j,t,p].X))
#                 print(XIijtp[i,j,t,p].X==int(XIijtp[i,j,t,p].X))

In [39]:
XMijtpL=[[[[0 for p in P]for t in T] for j in N]for i in M]
XIijtpL=[[[[0 for p in P]for t in T] for j in N]for i in M+S]

if m1.status == GRB.OPTIMAL:
        for i in M:
            for j in N:
                for t in T:
                    for p in P:
                        XMijtpL[i][j][t][p]=XMijtp[i,j,t,p].X
        for i in S:
            for j in N:
                for t in T:
                    for p in P:
                        XIijtpL[i][j][t][p]=XIijtp[i,j,t,p].X

In [40]:
D_v_jtp=[[[0 for p in P] for t in T] for j in N]
D_v_jtp2=[[[0 for p in P] for t in T] for j in N]

Vac_R_jtp=[[[0 for p in P] for t in T] for j in N]

for p in range(0, rv):
    for j in N:
        for t in T:
            D_v_jtp[j][t][p]=0

DZ=[254000136, 271572988, 168978824, 470760837, 45256788]
DZ=[254000136, 271572988, 156437522, 133630914, 337510504, 44876207, 12541302]

m2=gp.Model('IEV_M2')
xRMijtp=m2.addVars(M,N,T,P, vtype=GRB.CONTINUOUS, name='xRMijtp')
xRIijtp=m2.addVars(S,N,T,P, vtype=GRB.CONTINUOUS, name='xRIijtp')

TRijp1=m2.addVars(M+S,N,P, vtype=GRB.INTEGER, name='TRijp')
Yijrp1=m2.addVars(M+S,N,R,P, vtype=GRB.BINARY, name='Yijrp')
neta1=m2.addVars(N,F,vtype=GRB.BINARY, name='neta')

m2.addConstrs(gp.quicksum(XMijtpL[i][j][t][p] for j in N)+gp.quicksum(xRMijtp[i,j,t,p] for j in N)<=(MCap[p][i][t])-(Mexp[p][i][t]) for p in range(0,len(P)) for i in M for t in T)

m2.addConstrs(gp.quicksum(XIijtpL[i][j][t][p] for j in N)+gp.quicksum(xRIijtp[i,j,t,p] for j in N)<=SCap[p][i][t] for p in range(0,len(P)) for i in S for t in T)

m2.addConstrs(gp.quicksum(gp.quicksum(xRMijtp[i,j,t,p] for i in M for j in N)+gp.quicksum(xRIijtp[i,j,t,p] for i in S for j in N)for t in T)
              <=(f2+0.5)*(sum(sum(MCap[p][i][t]for t in T) for i in M)-sum(sum(Mexp[p][i][t]for t in T) for i in M)+sum(sum(SCap[p][i][t]for t in T)for i in S))for p in P)

m2.addConstrs(gp.quicksum(Yijrp1[i,j,r,p] for r in R)<=1 for i in M+S for j in N for p in range(0,len(P)))

DM2NP=[[0  for i in N] for p in P]
for j in N:
    for t in T:
        for p in P:
            if p>=rv:
                Vac_R_jtp[j][t][p]= gp.quicksum((1-rpd)**(l)*(gp.quicksum(xRMijtp[i,j,t,p-l]for i in M)+gp.quicksum(xRIijtp[i,j,t,p-l]for i in S))for l in range(min(p-rv,ex)))

wijrp1= m2.addVars(M+S, N, R, P, vtype=GRB.CONTINUOUS, name="wijrp")

for p in range(0,rv):
    for j in N:
        DM2NP[p][j]=DM2NPPP[p][j]

for j in N:
    for p in range(rv,len(P)):
        DM2NP[p][j] = DM2NP[p-1][j]-gp.quicksum(XMijtpL[i][j][t][p-1] for i in M for t in T)-gp.quicksum(XIijtpL[i][j][t][p-1] for i in S for t in T)

for j in N:
    for t in T:
        for p in P:
            if p<rv:
                Vac_avai_jtp[j][t][p]=gp.quicksum(((1-pd)**(p-k))*(gp.quicksum(XMijtpL[i][j][t][k]for i in M)+gp.quicksum(XIijtpL[i][j][t][k] for i in S)) for k in range(max(0,p-ex), p+1))
            if p>=rv:
                Vac_avai_jtp[j][t][p]=gp.quicksum(((1-pd)**(p-k))*(gp.quicksum(XMijtpL[i][j][t][k] for i in M)+gp.quicksum(XIijtpL[i][j][t][k] for i in S)) for k in range(max(0,p-ex), p+1))+gp.quicksum((1-rpd)**(l)*(gp.quicksum(xRMijtp[i,j,t,p-l]for i in M)+gp.quicksum(xRIijtp[i,j,t,p-l]for i in S))for l in range(min(p-rv+1,ex+1))) 

m2.addConstrs(gp.quicksum(Vac_avai_jtp[j][t][p] for t in T) <= gp.quicksum(CapR[f]*neta1[j,f] for f in F) for j in N for p in P)


for p in range(rv, len(P)):
    for j in N:
        for t in T:
            D_v_jtp[j][t][p]=pd*(gp.quicksum(((1-pd)**(p-rv-k))*(gp.quicksum(XMijtpL[i][j][t][k] for i in M)+gp.quicksum(XIijtpL[i][j][t][k] for i in S)) for k in range (max(0,p-rv-ex), (p-rv)+1)))+(D_v_jtp[j][t][p-1]-(gp.quicksum(xRMijtp[i,j,t,p-1] for i in M)+gp.quicksum(xRIijtp[i,j,t,p-1] for i in S)))
            
    PctPV=[[[0 for t in T] for j in N] for p in P]
    b=[]
    a=[]

for p in P:
    for j in N:
        for t in T:
            if p>=rv:
                D_v_jtp[j][t][p]=pd*(sum(((1-pd)**(p-rv-k))*(sum(XMijtpL[i][j][t][k] for i in M)+sum(XIijtpL[i][j][t][k] for i in S)) for k in range (max(0,p-rv-ex), (p-rv)+1)))
                D_v_jtp2[j][t][p]=D_v_jtp[j][t][p-1]-(gp.quicksum(xRMijtp[i,j,t,p-1] for i in M)+gp.quicksum(xRIijtp[i,j,t,p-1] for i in S))

                if D_v_jtp[j][t][p]!=0: 
                    aa=(0.999999*((f2*(sum(MCap[p][i][t] for i in M)-sum(Mexp[p][i][t] for i in M)+sum(SCap[p][i][t] for i in S))/(D_v_jtp[j][t][p] ))))
                    if aa>=1:
                        PctPV[p][j][t]=1
                    else:
                        PctPV[p][j][t]=aa
            
m2.addConstrs(gp.quicksum(xRMijtp[i,j,t,p] for i in M) + gp.quicksum(xRIijtp[i,j,t,p] for i in S)>= ma.ceil(0.07*D_v_jtp[j][t][p]) for p in range(rv, len(P)) for j in N for t in T)
m2.addConstrs(gp.quicksum(xRMijtp[i,j,t,p] for i in M) + gp.quicksum(xRIijtp[i,j,t,p] for i in S)==0 for p in range(0, rv) for j in N for t in T)

m2.addConstrs(wijrp1[i,j,r,p]>=TRijp1[i,j,p]-200*(1-Yijrp1[i,j,r,p]) for r in R for j in N for i in M+S for p in range(0,len(P)))
m2.addConstrs(gp.quicksum(wijrp1[i,j,r,p] for r in R) <=TRijp1[i,j,p] for p in P for i in M+S for j in N)

m2.addConstrs(wijrp1[i,j,r,p]<=1200* Yijrp1[i,j,r,p] for p in range(0,len(P)) for i in M+S for j in N for r in R)

m2.addConstrs(TRijp1[i,j,p]<=sum(xRMijtp[i,j,t,p]+XMijtpL[i][j][t][p] for t in T) for p in P for i in M for j in N)
m2.addConstrs(TRijp1[i,j,p]<=sum(xRIijtp[i,j,t,p]+XIijtpL[i][j][t][p] for t in T) for p in P for i in S for j in N)

m2.addConstrs(gp.quicksum(PCapr[r]*wijrp1[i,j,r,p] for r in R)-gp.quicksum((XMijtpL[i][j][t][p])+(xRMijtp[i,j,t,p]) for t in T)*0.005 >=0 for j in N for i in M for p in range(0,len(P)))
m2.addConstrs(gp.quicksum(PCapr[r]*wijrp1[i,j,r,p] for r in R)-gp.quicksum((XIijtpL[i][j][t][p])+(xRIijtp[i,j,t,p]) for t in T )*0.005 >=0 for j in N for i in S for p in range(0,len(P)))


m2.addConstrs(gp.quicksum(PCapr[r]*wijrp1[i,j,r,p] for r in R)-gp.quicksum((xRMijtp[i,j,t,p]) for t in T)*0.005 >=0 for j in N for i in M for p in range(0,len(P)))
m2.addConstrs(gp.quicksum(PCapr[r]*wijrp1[i,j,r,p] for r in R)-gp.quicksum((xRIijtp[i,j,t,p]) for t in T )*0.005 >=0 for j in N for i in S for p in range(0,len(P)))

m2.addConstrs(ma.ceil(0.9999999*(f2)*(sum(sum(MCap[p][i][t]for t in T) for i in M)-sum(sum(Mexp[p][i][t]for t in T) for i in M)+sum(sum(SCap[p][i][t]for t in T) for i in S)))>= gp.quicksum(D_v_jtp[j][t][p] for t in T) for p in range(rv, len(P)) for j in N)    


{(2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (4, 0): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (4, 2): <gurobi.Constr *Awaiting Model Update*>,
 (4, 3): <gurobi.Constr *Awaiting Model Update*>,
 (4, 4): <gurobi.Constr *Awaiting Model Update*>}

In [41]:
m2.update()

TC2=gp.quicksum(2*dM2TN[i][j][r]*Cost[r]*TRijp1[i,j,p]for p in range(0,len(P)) for r in R for j in N for i in M+S)

MC1=gp.quicksum(neta1[j,f]*CF[j][f] for j in N for f in F)
HC1=gp.quicksum(Vac_avai_jtp[j][t][p]-
               (gp.quicksum(XMijtpL[i][j][t][p] for i in M)+
                gp.quicksum(XIijtpL[i][j][t][p] for i in S)+
                gp.quicksum(xRMijtp[i,j,t,p] for i in M)+
                gp.quicksum(xRIijtp[i,j,t,p] for i in S)
               )for j in N for t in T for p in P)
CH1=gp.quicksum(Cost[r]*Yijrp1[i,j,r,p] for p in P for i in M+S for j in N for r in R)
UD=gp.quicksum(D_v_jtp[j][t][p-1]-(gp.quicksum(xRMijtp[i,j,t,p-1] for i in M)+gp.quicksum(xRIijtp[i,j,t,p-1] for i in S)) for p in range(1,len(P)) for j in N for t in T)

m2.setObjective(TC2+MC1+HC1+CH1+UD, GRB.MINIMIZE)

m2.Params.Heuristics=0
m2.Params.Presolve=0
m2.Params.Aggregate=0
m2.setParam("IntegralityFocus",1)
m2.setParam("IntFeasTol",1e-09)
m2.setParam("MIPGap", 0.0001)

m2.optimize()


Set parameter Heuristics to value 0
Set parameter Presolve to value 0
Set parameter Aggregate to value 0
Set parameter IntegralityFocus to value 1
Set parameter IntFeasTol to value 1e-09
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 5800HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 1325 rows, 1115 columns and 5855 nonzeros
Model fingerprint: 0xe1cfd6df
Variable types: 700 continuous, 415 integer (315 binary)
Coefficient statistics:
  Matrix range     [5e-03, 4e+08]
  Objective range  [2e-01, 2e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+08]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Variable types: 700 continuous, 415 integer (315 binary)

Root relaxation: objective 3.501974e+08, 426 iterations, 0.01 seconds (0.00 work units)

 

In [42]:
def extract_and_display_results(m, P):
    # Dictionary to hold results for each period
    results_by_period = {p: [] for p in P}
    
    # Extract and sort results by period
    for var in m.getVars():
        var_name = var.VarName
        var_value = var.X
        if var_value > 0:  # Filter out variables with value <= 0
            try:
                # Extract the indices from the variable name
                parts = var_name.split('[')
                indices = parts[-1].split(',')
                if len(indices) == 4:
                    p = int(indices[-1][:-1])  # 'p' is the last index
                elif len(indices) == 5:
                    p = int(indices[-2])  # 'p' is the second-to-last index
                results_by_period[p].append((var_name, var_value))
            except Exception as e:
                print(f"Error parsing variable name '{var_name}': {e}")
    
    for p in P:
        if results_by_period[p]:
            print(f"Results for period p = {p}:")
            for var_name, var_value in results_by_period[p]:
                print(f"{var_name}: {var_value}")
            print("\n")
        else:
            print(f"No results for period p = {p}\n")


In [43]:
extract_and_display_results(m1, P)

Results for period p = 0:
XMijtp[0,0,0,0]: 48938729.0
XMijtp[0,1,0,0]: 35840084.0
XMijtp[0,2,0,0]: 33221187.0
XMijtp[1,1,1,0]: 16551086.0
XMijtp[1,3,1,0]: 92551442.0
XMijtp[1,4,1,0]: 8897472.0
XIijtp[2,1,2,0]: 1000000.0
XIijtp[3,0,3,0]: 997623.0
Yijrp[0,0,0,0]: 1.0
Yijrp[0,1,0,0]: 1.0
Yijrp[0,2,0,0]: 1.0
Yijrp[1,1,0,0]: 1.0
Yijrp[1,3,0,0]: 1.0
Yijrp[1,4,0,0]: 1.0
Yijrp[2,1,1,0]: 1.0
Yijrp[3,0,0,0]: 1.0
wijrp[0,0,0,0]: 1.0
wijrp[0,1,0,0]: 1.0
wijrp[0,2,0,0]: 1.0
wijrp[1,1,0,0]: 1.0
wijrp[1,3,0,0]: 1.0
wijrp[1,4,0,0]: 1.0
wijrp[2,1,1,0]: 1.0
wijrp[3,0,0,0]: 1.0


Results for period p = 1:
XMijtp[0,0,0,1]: 48938730.0
XMijtp[0,1,0,1]: 35840083.0
XMijtp[0,2,0,1]: 33221187.0
XMijtp[1,1,1,1]: 16551087.0
XMijtp[1,3,1,1]: 92551442.0
XMijtp[1,4,1,1]: 8897471.0
XIijtp[2,1,2,1]: 1000000.0
XIijtp[3,0,3,1]: 997622.0
TRijp[0,0,0]: 1.0
TRijp[0,0,1]: 1.0
TRijp[0,1,0]: 1.0
TRijp[0,1,1]: 1.0
TRijp[0,1,2]: 1.0
TRijp[0,1,3]: 1.0
TRijp[0,1,4]: 1.0
TRijp[0,2,0]: 1.0
TRijp[0,2,1]: 1.0
TRijp[0,2,2]: 1.0
TRijp[

In [44]:
extract_and_display_results(m2, P)

Results for period p = 0:
Yijrp[0,0,0,0]: 1.0
Yijrp[0,1,0,0]: 1.0
Yijrp[0,2,0,0]: 1.0
Yijrp[1,1,0,0]: 1.0
Yijrp[1,3,0,0]: 1.0
Yijrp[1,4,0,0]: 1.0
Yijrp[2,1,1,0]: 1.0
Yijrp[3,0,1,0]: 1.0
wijrp[0,0,0,0]: 1.0
wijrp[0,1,0,0]: 1.0
wijrp[0,2,0,0]: 1.0
wijrp[1,1,0,0]: 1.0
wijrp[1,3,0,0]: 1.0
wijrp[1,4,0,0]: 1.0
wijrp[2,1,1,0]: 1.0
wijrp[3,0,1,0]: 1.0


Results for period p = 1:
Yijrp[0,0,0,1]: 1.0
Yijrp[0,1,0,1]: 1.0
Yijrp[0,2,0,1]: 1.0
Yijrp[1,1,0,1]: 1.0
Yijrp[1,3,0,1]: 1.0
Yijrp[1,4,0,1]: 1.0
Yijrp[2,1,1,1]: 1.0
Yijrp[3,0,1,1]: 1.0
wijrp[0,0,0,1]: 1.0
wijrp[0,1,0,1]: 1.0
wijrp[0,2,0,1]: 1.0
wijrp[1,1,0,1]: 1.0
wijrp[1,3,0,1]: 1.0
wijrp[1,4,0,1]: 1.0
wijrp[2,1,1,1]: 1.0
wijrp[3,0,1,1]: 1.0


Results for period p = 2:
xRMijtp[0,0,0,2]: 3000000.0
xRMijtp[0,1,0,2]: 3982649.0000219494
xRMijtp[0,2,0,2]: 85033642.99997805
xRMijtp[1,1,1,2]: 926861.0
xRMijtp[1,3,1,2]: 5182881.0
xRMijtp[1,4,1,2]: 66474677.0
xRIijtp[2,1,2,2]: 1000000.0
xRIijtp[3,0,3,2]: 1000000.0
Yijrp[0,0,1,2]: 1.0
Yijrp[0,1,0,2]: 1

In [45]:
for p in range(0,len(P)):
    for t in T:
        for j in N:
            for i in M:
                if xRMijtp[i,j,t,p].X!=int(xRMijtp[i,j,t,p].X):
                    print('xRM',i,j,t,p) 
                    print(xRMijtp[i,j,t,p].X-int(xRMijtp[i,j,t,p].X))
                print(xRMijtp[i,j,t,p].X==int(xRMijtp[i,j,t,p].X))

                
for p in range(0,len(P)):
    for t in T:
        for j in N:
            for i in S:
                if xRIijtp[i,j,t,p].X!=int(xRIijtp[i,j,t,p].X) :
                    print('xRI', i,j,t,p) 
                    print(xRIijtp[i,j,t,p].X-int(xRIijtp[i,j,t,p].X))
                print(xRIijtp[i,j,t,p].X==int(xRIijtp[i,j,t,p].X))

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
xRM 0 1 0 2
2.1949410438537598e-05
False
True
xRM 0 2 0 2
0.9999780505895615
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
xRM 0 1 0 3
9.414553642272949e-05
False
True
xRM 0 2 0 3
0.9999058544635773
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
xRM 0 0 0 4
0.9999971389770508
False
True
xRM 0 1 0 4
0.999996900